In [75]:
import random
import os
import torch
import numpy as np
import json
import cv2
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
#import pycocotools.mask as mask_util
import torch
from torch import device

#from detectron2.layers.roi_align import ROIAlign
#from detectron2.utils.memory import retry_if_cuda_oom

In [76]:
BATCH_SIZE = 8
IMAGE_SIZE = (1716, 942)
root = os.path.join('.')

In [93]:
def read_masks(filepath='./SEG_Train_Datasets/Train_Annotations'):
    '''
    Read masks from directory and tranform to categorical
    '''
    file_list = [file for file in os.listdir(filepath) if file.endswith('.json')]
    file_list.sort()
    n_masks = len(file_list)
    masks = np.empty((n_masks, 1716, 942), dtype=bool)

    for i, file in enumerate(file_list):
        with open(filepath+'/'+file, newline='') as jsonfile:
            data = json.load(jsonfile)
            for i in range(len(data['shapes'])):
                for j in range(len(data['shapes'][i]['points'])):
                    #print(int(data['shapes'][i]['points'][j][0]), int(data['shapes'][i]['points'][j][1]))
                    masks[i, int(data['shapes'][i]['points'][j][0]), int(data['shapes'][i]['points'][j][1])] = 1


    return masks

In [43]:
#test
'''
with open('./SEG_Train_Datasets/Train_Annotations/00001002.json', newline='') as jsonfile:
    data = json.load(jsonfile)
    masks = np.empty((1716, 942))
    masks[int(data['shapes'][1]['points'][0][0]), int(data['shapes'][1]['points'][0][1])] = 1
    #print(data)
    #print(len(data['shapes']))
    #print(data['shapes'][1]['points'][0][0])
    #print(len(data['shapes'][1]['points']))
    #print(masks)
    for i in range(len(data['shapes'])):
        for j in range(len(data['shapes'][i]['points'])):
            #print(int(data['shapes'][i]['points'][j][0]), int(data['shapes'][i]['points'][j][1]))
            masks[int(data['shapes'][i]['points'][j][0]), int(data['shapes'][i]['points'][j][1])] = 1

 

    print(masks)
'''

"\nwith open('./SEG_Train_Datasets/Train_Annotations/00001002.json', newline='') as jsonfile:\n    data = json.load(jsonfile)\n    masks = np.empty((1716, 942))\n    masks[int(data['shapes'][1]['points'][0][0]), int(data['shapes'][1]['points'][0][1])] = 1\n    #print(data)\n    #print(len(data['shapes']))\n    #print(data['shapes'][1]['points'][0][0])\n    #print(len(data['shapes'][1]['points']))\n    #print(masks)\n    for i in range(len(data['shapes'])):\n        for j in range(len(data['shapes'][i]['points'])):\n            #print(int(data['shapes'][i]['points'][j][0]), int(data['shapes'][i]['points'][j][1]))\n            masks[int(data['shapes'][i]['points'][j][0]), int(data['shapes'][i]['points'][j][1])] = 1\n\n \n\n    print(masks)\n"

In [82]:
class SegDs(Dataset):
    def __init__(self, root='.', train_test='train', is_val=False, train_val_split=False, val_split_rate=0.9,
                 transform=None):
        self.filenames = []
        self.labels = np.array([])
        self.len = 0
        self.transform = transform
        self.train_test = train_test
        if train_test == 'test':
            file_path = os.path.join(root, 'Public_Image')
        else:
            file_path = os.path.join(root,  'SEG_Train_Datasets', 'Train_Images')
            #print('file path is', file_path)
            #         label
            if os.path.isfile(train_test + '_label_cat.npy'):
                self.labels = np.load(train_test + '_label_cat.npy')
                print(self.labels.shape)
            else:
                self.labels = read_masks()
                # print(type(self.labels))
                print(self.labels.shape)
                np.save(train_test + '_label_cat.npy', self.labels)
            # data
        self.filenames = [os.path.join(file_path, file) for file in os.listdir(file_path) if file.endswith('.jpg')]

        self.filenames.sort()
        if train_val_split:
            split_nums = int(len(self.filenames) * val_split_rate)
            self.filenames = self.filenames[split_nums:] if is_val else self.filenames[:split_nums]
            self.labels = self.labels[split_nums:] if is_val else self.labels[:split_nums]
        self.len = len(self.filenames)
        # print(self.filenames[808])

    def __getitem__(self, index):
        image_fn = self.filenames[index]
        image = Image.open(image_fn)

        if self.transform is not None:
            image = self.transform(image)

        if self.train_test == 'test':
            return image
        else:
            label = self.labels[index]
            return image, label

    def __len__(self):
        return self.len

In [80]:
use_cuda = torch.cuda.is_available()
print(torch.__version__)
torch.manual_seed(123)
device = torch.device("cuda:0" if use_cuda else "cpu")
print('Device used:', device)

1.10.2
Device used: cuda:0


In [94]:
train_set = SegDs(train_test='train', is_val=False, train_val_split=False, transform=transforms.Compose([
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
]))

(1053, 1716, 942)
